### Часть 2
* Загрузим подготовленный набор обучающих данных из части 1.
* Подготовим тестовый набор, проведем обогащение, и приведем к тому же набору атрибутов, что и обучающие.
* Предскажем требуемые значения и выгрузим результаты.


### Подключение библиотек

In [105]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from transliterate import translit
from tqdm import notebook
from sklearn_som.som import SOM


### Необходимые вспомогательные функции

In [106]:
# вспомогательная функция позволяющая обогатить данные 
def data_preproccesing (data):
# add total items per day
    data_day_count = data.groupby("day").count()["build_year"]
    data["day_count"] = data["day"].apply(lambda x:data_day_count.loc[x])
# approximate values (clean-up)
    data.loc[data.build_year == 0, 'build_year'] = np.NaN
    data['build_year'] = data['build_year'].fillna((data.groupby(['building_series_id'])['build_year'].transform('median')))
    data.loc[data['build_year'].isna(), 'build_year'] = data['build_year'].mean()
    data['build_year'] = data['build_year'].astype(np.uint16)
    if 'has_elevator' in data.columns:
# elevator for 6+ floors
        data.loc[(data.has_elevator==0) & (data.floor>5), 'has_elevator'] = 1
# fix living area
    data.loc[data.living_area == 0, 'living_area'] = np.NaN
    data['living_area'] = data['living_area'].fillna((data.groupby(['rooms'])['living_area'].transform('median')))
# fix price
    data.loc[data.price<100, 'price'] *= 1000  # цена в тыс.
    data.loc[data.price<1000, 'price'] *= 60  # цена в долларах
    if 'floors_total' in data.columns:
# fix celing height  # правим высоту потолков, если данных по высоте серии дома, берем среднее
        data.loc[(data.ceiling_height<2) | (data.ceiling_height>5), 'ceiling_height'] = np.NaN
        data['ceiling_height'] = data['ceiling_height'].fillna(data.groupby(['building_series_id'])['ceiling_height'].transform('median'))
        data.loc[data['ceiling_height'].isna(), 'ceiling_height'] = data['ceiling_height'].mean()
# enrich data, % floor  # этажи как процент этажности
        data['floor'] = data['floor'] / data["floors_total"]
# locality, village/region/moscow/metro  # обогащение по типу города, нас пункту и т.п.
    if 'locality_name' in data.columns:
        data['loctype_village'] = (data['locality_name'].str.match(pat = 'городок|деревня|ДНП|поселок|посёлок|село|СНТ|товарищество|хутор')).astype(np.uint8)
        data['loctype_moscow'] = (data.locality_name == 'Москва').astype(np.uint8)
        data['loctype_region'] = ((data.loctype_village == 0) & (data.loctype_moscow == 0)).astype(np.uint8)
    if "site_id" in data.columns:  # удаляем часть строковых переменных (которые не имеет  смысла переводить в единичные вектора, они не дадут ни какого полезного результата)
        data = data.drop(['site_id', 'main_image', 'area', 'building_id', 'unified_address'], axis=1)
    if 'target_string' in data.columns:
        data = data.drop(['target_string'], axis=1)  # строковое описание таргета
# processing date
    if 'day' in data.columns:  # обогащение по времени 
        data['day'] = pd.to_datetime(data['day'])   # Series.dt.isocalendar().week
        data['year'] = data['day'].dt.year
        data['month'] = data['day'].dt.month
        data['week'] = data['day'].dt.isocalendar().week  #.dt.week
        data['dow'] = data['day'].dt.dayofweek
        data['dom'] = data['day'].dt.day
        data['doy'] = data['day'].dt.dayofyear
        data = data.drop(["day"], axis=1)
# adding holydays, 1-7 Jan, 8 Mar, 1 May, 9 May, 12 Jun, 4 Nov
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2017/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2018/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2019/
# http://www.consultant.ru/law/ref/calendar/proizvodstvennye/2020/
        # добавление выходных
        data['is_holyday'] = ((data['year'] == 2017 &
                                (((data['dom'] > 0) & (data['dom'] < 8) & data['month'] == 1) | 
                                (((data['dom'] == 23) | data['dom'] == 24)) & (data['month'] == 2)) |
                                ((data['dom'] == 8) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 8) | (data['dom'] == 9)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 6) & (data['month'] == 11))) |
                              ((data['year'] == 2018) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 23) & (data['month'] == 2)) |
                                (((data['dom'] == 8) | (data['dom'] == 9)) & (data['month'] == 3)) |
                                ((data['dom'] == 30) & (data['month'] == 4)) |
                                (((data['dom'] == 1) | (data['dom'] == 2) | (data['dom'] == 9)) & data['month'] == 5) |
                                (((data['dom'] == 11) | (data['dom'] == 12)) & (data['month'] == 6)) |
                                ((data['dom'] == 5) & (data['month'] == 11)) |
                                ((data['dom'] == 31) & (data['month'] == 12)))) |
                              ((data['year'] == 2019) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 8) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 2) | (data['dom'] == 3) | (data['dom'] == 9) | (data['dom'] == 10)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 4) & (data['month'] == 11)))) |
                              ((data['year'] == 2020) &
                                (((data['dom'] > 0) & (data['dom'] < 9) & data['month'] == 1) | 
                                ((data['dom'] == 24) & (data['month'] == 2)) |
                                ((data['dom'] == 9) & (data['month'] == 3)) |
                                (((data['dom'] == 1) | (data['dom'] == 4) | (data['dom'] == 5) | (data['dom'] == 11)) & data['month'] == 5) |
                                ((data['dom'] == 12) & (data['month'] == 6)) |
                                ((data['dom'] == 4) & (data['month'] == 11))))).astype(np.uint8)
# one-hot vectors перевод данных в единичные вектора
    if 'year' in data.columns:
        for label in ['year', 'month', 'week', 'dow', 'doy', 'dom', 'renovation',
                      'balcony', 'building_type', 'parking', 'floors_total', 'locality_name']:
            for l in data[label].unique():  # транслитим русские названия 
                data[label + "_" + translit(str(l), "ru", reversed=True)] = (data[label] == l).astype(np.uint8)
# boolean -> int  
    if 'studio' in data.columns:
        for label in ['studio', 'has_elevator', 'expect_demolition', 'is_apartment']:
            data[label] = data[label].astype(np.uint8)
# index (remove id from columns) 
    if 'id' in data.columns:
        data = data.set_index(['id'])
    return data

In [ ]:
# функция возвращает отношение текущей цены к медианной цене по району (метро) или по городу
def calc_price (data, group="", label=""):
    if data[group] in price_groups[group][label]:
        return data["price"] / price_groups[group][label][data[group]]
    else:
        return 1

### Загрузка данных

In [108]:
train_data = pd.read_csv('exposition_train.basic.csv.gz')
train_data = train_data.drop(labels=["doy_108"], axis=1)
train_data.head()

,total_area,ceiling_height,rooms,living_area,price,day_mean,price_locality_name_median,target
0,105.000000,3.00,3,50.000000,95000,2.456912,2.261905,1
1,40.000000,3.00,1,19.200001,25000,3.028689,1.000000,2
2,37.599998,2.64,0,19.000000,26000,3.091993,0.619048,2
3,80.000000,3.00,3,49.000000,35000,3.101010,1.250000,2
4,100.000000,3.00,3,49.000000,80000,2.495468,1.904762,3


### Нормализация данных
Проведем нормализацию при помощи StandardScaler всех значений исключив из данных target

In [109]:
scaler = StandardScaler()

In [110]:
train_data_mm = pd.DataFrame(scaler.fit_transform(train_data[train_data.columns[:-1]]))

In [111]:
train_data_mm.head()

,0,1,2,3,4,5,6
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,0.278301
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.132179
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.256098
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.050858
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,0.162128


In [112]:
train_data_mm.columns

RangeIndex(start=0, stop=7, step=1)

### SOM
Сформируем 5 моделей SOM с разными np.random.seed и разной сеткой, и получим кластеры по ним.

( ! ) Предупреждение.
* К сожалению сетку более чем 10 на 10 (т.е. более 100 кластеров) использовать не получается, так как обучающие данные распределяются не по всем кластерам и если тестовые данные распределяются по отличным от обучающих данных кластерам, невозможно сопоставить прогноз класса по кластерам обучающей выборки с тестовой. 

Для решения этой проблемы есть несколько вариантов, один из них выглядит как подстановка среднего значения класса если кластер не был найден.

Воспользуемся этим решением.

In [113]:
np.random.seed(32)
som0 = SOM(m=40, n=40, dim=len(train_data_mm.columns), max_iter=1000)
som0.fit(np.array(train_data_mm), epochs=100, shuffle=False)

In [114]:
np.random.seed(42)
som1 = SOM(m=45, n=45, dim=len(train_data_mm.columns), max_iter=1000)
som1.fit(np.array(train_data_mm), epochs=100, shuffle=False)

In [115]:
np.random.seed(77)
som2 = SOM(m=50, n=50, dim=len(train_data_mm.columns), max_iter=1000)
som2.fit(np.array(train_data_mm), epochs=100, shuffle=False)

In [116]:
np.random.seed(5)
som3 = SOM(m=60, n=60, dim=len(train_data_mm.columns), max_iter=1000)
som3.fit(np.array(train_data_mm), epochs=100, shuffle=False)

In [117]:
np.random.seed(89)
som4 = SOM(m=65, n=65, dim=len(train_data_mm.columns), max_iter=1000)
som4.fit(np.array(train_data_mm), epochs=100, shuffle=False)

Соберем кластеры по всем моделям в список labels_SOM

In [118]:
labels_SOM = []
model_SOM = [som0, som1, som2, som3, som4]
for i in range (5):
    print("Processing model: ", i)
    labels_SOM.append(model_SOM[i].predict(np.array(train_data_mm)))
labels_SOM


Processing model:  0
Processing model:  1
Processing model:  2
Processing model:  3
Processing model:  4


[array([1449, 1226,  961, ...,  536,  256,  712], dtype=int64),
 array([1420, 1386,   13, ..., 1207, 1747, 1165], dtype=int64),
 array([1365, 2467, 1254, ..., 1440,  804, 1248], dtype=int64),
 array([ 758, 3476, 3089, ..., 1957, 2815, 2198], dtype=int64),
 array([4200,  924, 2348, ..., 1892, 3782, 2832], dtype=int64)]

Добавим кластеры к обучающей выборке

In [119]:
for i in range (5):
    train_data_mm["label"+str(i)] = labels_SOM[i]
train_data_mm["target"] = train_data["target"]

In [120]:
train_data_mm.head()

,0,1,2,3,4,5,6,label0,label1,label2,label3,label4,target
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,0.278301,1449,1420,1365,758,4200,1
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.132179,1226,1386,2467,3476,924,2
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.256098,961,13,1254,3089,2348,2
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.050858,481,505,1379,1080,1700,2
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,0.162128,1449,1420,891,1402,4200,3


In [121]:
# train_data_mm.to_csv('train_data_mm_5_SOM_clusters.csv')

### Подготовка тестовых данных 
Проведем обогащение тестовых данных

In [122]:
test_data = pd.read_csv('exposition_test.tsv.gz', sep='\t')
test_data.head()

,building_series_id,site_id,parking,build_year,expect_demolition,main_image,latitude,total_area,ceiling_height,rooms,...,kitchen_area,day,public,longitude,price,flats_count,building_type,balcony,locality_name,renovation
0,663294,0,UNKNOWN,1971,False,//avatars.mds.yandex.net/get-realty/1900763/ad...,55.795704,36.000000,2.64,1,...,0.0,2020-01-25,True,37.602478,40000,80,PANEL,UNKNOWN,Москва,UNKNOWN
1,712125,0,UNKNOWN,1986,False,//avatars.mds.yandex.net/get-realty/1583116/ad...,55.605583,40.000000,2.48,1,...,10.0,2019-11-19,True,37.743679,25000,222,PANEL,LOGGIA,Москва,COSMETIC_DONE
2,0,0,UNKNOWN,2014,False,//avatars.mds.yandex.net/get-realty/2124710/ad...,55.925560,25.000000,0.00,0,...,0.0,2020-01-11,True,37.862965,19000,179,MONOLIT,LOGGIA,Королёв,COSMETIC_DONE
3,0,0,UNKNOWN,2001,False,//avatars.mds.yandex.net/get-realty/2958378/ad...,55.432522,42.000000,0.00,1,...,10.0,2020-01-27,True,37.544224,20000,0,PANEL,LOGGIA,Подольск,COSMETIC_DONE
4,1564812,0,UNKNOWN,2019,False,//avatars.mds.yandex.net/get-realty/2732616/ad...,55.917530,73.300003,2.80,3,...,10.2,2020-03-04,False,37.411098,68000,0,MONOLIT,TWO_LOGGIA,Химки,EURO


Для добавления зависимости срока экспозиции от количества объявлений в день (day_mean) в тестовые данные, построим предсказания на основе линейной регрессии обученной на exposition_train данных.

In [ ]:
train = pd.read_csv('exposition_train.tsv.gz', sep='\t')
train_day_count = train.groupby("day").count()["target"]
train_day_mean = train.groupby("day").mean()["target"]
train["day_count"] = train["day"].apply(lambda x:train_day_count.loc[x])
train["day_mean"] = train["day"].apply(lambda x:train_day_mean.loc[x])

Построим модель линейной регрессии 

In [124]:
x = np.array(train[train["day_count"]>max(train_day_count)-1]["day_count"]).reshape(-1, 1)
y = train[train["day_count"]>max(train_day_count)-1]["day_mean"]
day_model = LinearRegression().fit(x, y)

In [125]:
test_day_count = test_data.groupby("day").count()["total_area"]
test_data["day_count"] = test_data["day"].apply(lambda x:test_day_count.loc[x])
test_data["day_mean"] = day_model.predict(np.array(test_data["day_count"]).reshape(-1, 1))
test_data.loc[test_data["day_count"]<max(test_data["day_count"]),"day_mean"] = train_day_mean.mean()

In [126]:
test_data.head(10)

,building_series_id,site_id,parking,build_year,expect_demolition,main_image,latitude,total_area,ceiling_height,rooms,...,public,longitude,price,flats_count,building_type,balcony,locality_name,renovation,day_count,day_mean
0,663294,0,UNKNOWN,1971,False,//avatars.mds.yandex.net/get-realty/1900763/ad...,55.795704,36.000000,2.64,1,...,True,37.602478,40000,80,PANEL,UNKNOWN,Москва,UNKNOWN,352,2.931129
1,712125,0,UNKNOWN,1986,False,//avatars.mds.yandex.net/get-realty/1583116/ad...,55.605583,40.000000,2.48,1,...,True,37.743679,25000,222,PANEL,LOGGIA,Москва,COSMETIC_DONE,553,2.931129
2,0,0,UNKNOWN,2014,False,//avatars.mds.yandex.net/get-realty/2124710/ad...,55.925560,25.000000,0.00,0,...,True,37.862965,19000,179,MONOLIT,LOGGIA,Королёв,COSMETIC_DONE,381,2.931129
3,0,0,UNKNOWN,2001,False,//avatars.mds.yandex.net/get-realty/2958378/ad...,55.432522,42.000000,0.00,1,...,True,37.544224,20000,0,PANEL,LOGGIA,Подольск,COSMETIC_DONE,501,2.931129
4,1564812,0,UNKNOWN,2019,False,//avatars.mds.yandex.net/get-realty/2732616/ad...,55.917530,73.300003,2.80,3,...,False,37.411098,68000,0,MONOLIT,TWO_LOGGIA,Химки,EURO,464,2.931129
5,1564812,0,UNKNOWN,1961,False,//avatars.mds.yandex.net/get-realty/1651606/ad...,55.677845,32.000000,0.00,1,...,True,37.564484,40000,112,BRICK,BALCONY,Москва,COSMETIC_DONE,553,2.931129
6,1564812,0,UNKNOWN,2016,False,//avatars.mds.yandex.net/get-realty/2355710/ad...,55.842464,30.000000,3.00,0,...,False,37.373302,45000,0,MONOLIT,UNKNOWN,Москва,DESIGNER_RENOVATION,5259,2.931129
7,1564812,0,UNKNOWN,1952,False,//avatars.mds.yandex.net/get-realty/2090636/ad...,55.779575,40.000000,3.20,1,...,False,37.706863,40000,115,BRICK,LOGGIA,Москва,EURO,5259,2.931129
8,1564812,0,UNKNOWN,1966,False,//avatars.mds.yandex.net/get-realty/2353363/ad...,55.765087,48.000000,2.70,2,...,True,37.657494,110000,68,PANEL,BALCONY,Москва,EURO,485,2.931129
9,663320,0,UNKNOWN,2005,False,//avatars.mds.yandex.net/get-realty/2771165/ad...,55.886742,52.000000,2.74,2,...,True,37.647129,40000,235,PANEL,BALCONY,Москва,COSMETIC_DONE,6779,1.392246


Воспользуемся функцией data_preproccesing, а так же добавим среднюю цену.

In [ ]:
test_data = data_preproccesing(test_data)
test_data.head()

In [128]:
price_data = pd.DataFrame(test_data[["locality_name", "price"]])
price_groups = {"locality_name": {
    "median": price_data.groupby(["locality_name"]).median()["price"]
}}

In [129]:
for group in price_groups:
    print ("Processing:", group, end=" ")
    for label in price_groups[group]:
        print (label, end=" ")
        test_data["price_" + group + "_" + label] = test_data.apply(calc_price, axis=1,
                                                                      group=group, label=label)
    print ("")

Processing: locality_name median 


Сформируем тестовые данные по самым значимым параметрам которые определили на обучающих данных и сохраним этот набор как базовый тестовый.

In [130]:
test_data = pd.DataFrame(test_data[['total_area', 'ceiling_height', 'rooms', 'living_area',
                                   'price', 'day_mean', 'price_locality_name_median']])

In [131]:
test_data.head()

,total_area,ceiling_height,rooms,living_area,price,day_mean,price_locality_name_median
id,,,,,,,
13762887891614807236,36.000000,2.64,1,19.000000,40000,2.931129,0.888889
14654451946329972059,40.000000,2.48,1,20.000000,25000,2.931129,0.555556
17449292585625593873,25.000000,2.70,0,12.000000,19000,2.931129,0.791667
15597282206699587329,42.000000,2.70,1,20.000000,20000,2.931129,0.869565
3718201047023531068,73.300003,2.80,3,45.799999,68000,2.931129,2.259136


Сохраним полученный набор как некоторый тестовый базовый

In [132]:
test_data.to_csv("exposition_test.basic.csv.gz", index=False)

### Формирование предсказания

Для формирования предсказаний проведем нормировку тестовых данных 

In [133]:
test_data = pd.read_csv('exposition_test.basic.csv.gz')

In [134]:
test_data_mm = pd.DataFrame(scaler.transform(test_data))
test_data_mm.head()

,0,1,2,3,4,5,6
0,-0.618155,-0.645106,-0.861151,-0.690542,-0.089888,0.005938,-0.168322
1,-0.483202,-1.427399,-0.861151,-0.635189,-0.217544,0.005938,-0.276751
2,-0.989277,-0.351746,-2.026671,-1.078012,-0.268606,0.005938,-0.199947
3,-0.415725,-0.351746,-0.861151,-0.635189,-0.260095,0.005938,-0.174608
4,0.640286,0.137186,1.469889,0.792913,0.148401,0.005938,0.277401


In [135]:
train_data_mm.head()

,0,1,2,3,4,5,6,label0,label1,label2,label3,label4,target
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,0.278301,1449,1420,1365,758,4200,1
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.132179,1226,1386,2467,3476,924,2
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.256098,961,13,1254,3089,2348,2
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.050858,481,505,1379,1080,1700,2
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,0.162128,1449,1420,891,1402,4200,3


Сформируем список кластеров для каждой модели и среднее значение target по группе кластера

In [136]:
clusters = []
for i in range(5):
    clusters.append(train_data_mm.groupby("label"+str(i)).mean()["target"])
clusters

[label0
 0       2.990291
 1       3.055556
 2       2.825000
 4       3.250000
 5       3.650000
           ...   
 1589    3.457143
 1591    3.222222
 1594    1.090909
 1597    3.512367
 1598    3.374613
 Name: target, Length: 812, dtype: float64,
 label1
 7       2.000000
 8       3.814815
 12      2.957494
 13      2.951685
 14      3.090909
           ...   
 2009    5.000000
 2010    4.000000
 2019    4.000000
 2023    2.722222
 2024    1.000000
 Name: target, Length: 894, dtype: float64,
 label2
 0       3.142857
 6       2.000000
 7       1.833333
 10      3.409091
 15      5.000000
           ...   
 2484    3.200000
 2490    3.000000
 2491    3.875000
 2495    1.000000
 2497    2.400000
 Name: target, Length: 1052, dtype: float64,
 label3
 0       3.462366
 5       1.932584
 9       3.700000
 12      2.000000
 13      3.829268
           ...   
 3589    2.958115
 3590    3.000000
 3593    1.818182
 3596    3.500000
 3598    3.911111
 Name: target, Length: 1223, dtype: float64

Предскажем значение кластера для тестовых данных

In [137]:
y = []
for i in range(5):
    print("Processing model: ", i)
    y.append(model_SOM[i].predict(np.array(test_data_mm)))


Processing model:  0
Processing model:  1
Processing model:  2
Processing model:  3
Processing model:  4


Сформируем серию данных из предсказанных кластеров

In [138]:
pred = pd.DataFrame(np.array(y).reshape(-1,5))
pred

,0,1,2,3,4
0,771,1091,1001,728,1430
1,1267,1109,403,553,216
2,1227,520,812,728,1185
3,1509,750,1471,1302,771
4,924,1266,1001,406,829
...,...,...,...,...,...
71661,2439,1147,924,3981,2690
71662,2801,3981,3981,788,3791
71663,1013,3958,3754,1156,1147
71664,2833,742,3762,926,3762


Преобразуем данные clusters (кластер - средний таргет в группе кластеров) в словарь (для сетки размером более 10x10, актуально)

In [139]:
forclusters_dict = []
for i in range(5):
    forclusters_dict.append(clusters[i].to_dict())
# forclusters_dict

Добавим к кластерам тестовой выборки предсказания по кластерам из обучающей

In [140]:
# for i in range(5):
#     clusters_dict = forclusters_dict[i]
#     print('Process' , i)
#     pred["pred"+str(i)] = pred[i].apply(lambda x: clusters_dict[x])

В следующем цикле сопоставляем предсказания срока экспозиции по кластерам обучающей выборки с кластерами тестовой, если тестовые данные попали в кластер которого нет в обучающих, тогда значение срока экспозиции берем как среднее равное 3.

In [141]:
for i in range(5):
    # count = 0
    clusters_dict = forclusters_dict[i]
    print('Processing' , i)
    pred["pred"+str(i)] = pred[i].apply(lambda x: clusters_dict[x] if clusters_dict.get(x) != None else 3)
    # print(count)

Processing 0
Processing 1
Processing 2
Processing 3
Processing 4


Сформируем итоговое предсказание ансамбля для тестовой выборки.

In [150]:
pred["prediction"] = (pred["pred0"]*0.2 + pred["pred1"]*0.2 + pred["pred2"]*0.2 +
                    pred["pred3"]*0.2 + pred["pred4"]*0.2)

In [151]:
pred["prediction"].head(10)

0    3.295890
1    2.918144
2    3.104410
3    3.072208
4    3.055710
5    3.074076
6    2.939640
7    2.977208
8    2.958354
9    3.081368
Name: prediction, dtype: float64

### Загрузка решения

Выгрузим решение в требуемом формате

In [152]:
submission = pd.read_csv('https://video.ittensive.com/machine-learning/hacktherealty/E/exposition_sample_submission.tsv', sep='\t')

In [155]:
submission['target'] = np.around(pred["prediction"])

In [156]:
submission["target"] = submission["target"].apply(lambda x:max(1, min(x,5))).astype(np.uint8)
submission.to_csv('submission_last.tsv', sep='\t', index=False)

In [157]:
# submission.head(15)